# Develop negative log-likelihood with scaled inputs

In [ ]:
from __future__ import annotations

import numdifftools as nd
import numpy as np

from numpy.typing import ArrayLike
from scipy.fft import rfft, irfft, rfftfreq
from scipy.optimize import minimize
from matplotlib import pyplot as plt
from matplotlib.figure import figaspect
from scipy.optimize import approx_fprime
from numpy.random import default_rng

import thztools as thz
from thztools.thztools import _costfun_noisefit as costfun

## Simulate measurements

In [ ]:
rng = np.random.default_rng(0)
n = 256
m = 64
ts = 0.05
thz.global_options.sampling_time = ts
t = thz.timebase(n)
mu = thz.wave(n)
sigma = np.array([1e-5, 1e-2, 1e-3])
noise_model = thz.NoiseModel(
    sigma_alpha=sigma[0], sigma_beta=sigma[1], sigma_tau=sigma[2]
)
noise = noise_model.noise((np.ones((m, 1)) * mu), seed=0)
x = np.array(mu + noise)
delta_mu = np.zeros(n)
delta_a = np.zeros(m - 1)
eta = np.zeros(m - 1)

logv = np.log(sigma**2)
scale_sigma_alpha = noise_model.sigma_alpha
scale_sigma_beta = noise_model.sigma_beta
scale_sigma_tau = noise_model.sigma_tau
scale_delta_mu = 1e-0 * noise_model.amplitude(mu)
scale_delta_a = 1e-4 * np.ones(m - 1)
scale_eta = 1e-3 * np.ones(m - 1)

## Check gradient

In [ ]:
_, grad_delta_mu_tdnll = costfun(
    x,
    logv[0],
    logv[1],
    logv[2],
    delta_mu,
    delta_a,
    eta / ts,
    fix_logv_alpha=True,
    fix_logv_beta=True,
    fix_logv_tau=True,
    fix_delta_mu=False,
    fix_delta_a=True,
    fix_eta=True,
    scale_sigma_alpha=1.0,
    scale_sigma_beta=1.0,
    scale_sigma_tau=1.0,
    scale_delta_mu=np.ones(n),
    scale_delta_a=np.ones(m - 1),
    scale_eta=np.ones(m - 1),
)

_, grad_delta_mu_costfun = costfun(
    x,
    np.log(np.exp(logv[0]) / scale_sigma_alpha**2),
    np.log(np.exp(logv[1]) / scale_sigma_beta**2),
    np.log(np.exp(logv[2]) / scale_sigma_tau**2),
    delta_mu / scale_delta_mu,
    delta_a,
    eta / ts,
    fix_logv_alpha=True,
    fix_logv_beta=True,
    fix_logv_tau=True,
    fix_delta_mu=False,
    fix_delta_a=True,
    fix_eta=True,
    scale_sigma_alpha=scale_sigma_alpha,
    scale_sigma_beta=scale_sigma_beta,
    scale_sigma_tau=scale_sigma_tau,
    scale_delta_mu=scale_delta_mu,
    scale_delta_a=np.ones(m - 1),
    scale_eta=np.ones(m - 1),
)

grad_delta_mu_nd = nd.Gradient(
    lambda _delta_mu: costfun(
        x,
        logv[0],
        logv[1],
        logv[2],
        _delta_mu,
        delta_a,
        eta / ts,
        fix_logv_alpha=True,
        fix_logv_beta=True,
        fix_logv_tau=True,
        fix_delta_mu=True,
        fix_delta_a=True,
        fix_eta=True,
        scale_sigma_alpha=1.0,
        scale_sigma_beta=1.0,
        scale_sigma_tau=1.0,
        scale_delta_mu=np.ones(n),
        scale_delta_a=np.ones(m - 1),
        scale_eta=np.ones(m - 1),
    )[0],
    step=1e-6,
)(delta_mu)

np.stack(
    (
        grad_delta_mu_tdnll,
        grad_delta_mu_costfun / scale_delta_mu,
        grad_delta_mu_nd,
    )
).T

In [ ]:
plt.plot(t, grad_delta_mu_tdnll)
plt.plot(t, grad_delta_mu_costfun / scale_delta_mu)
plt.show()
plt.plot(t, grad_delta_mu_tdnll - grad_delta_mu_costfun / scale_delta_mu)
plt.show()

In [ ]:
val, grad_logv_tdnll = costfun(
    x,
    logv[0],
    logv[1],
    logv[2],
    delta_mu,
    delta_a,
    eta / ts,
    fix_logv_alpha=False,
    fix_logv_beta=False,
    fix_logv_tau=False,
    fix_delta_mu=True,
    fix_delta_a=True,
    fix_eta=True,
    scale_sigma_alpha=1.0,
    scale_sigma_beta=1.0,
    scale_sigma_tau=1.0,
    scale_delta_mu=scale_delta_mu,
    scale_delta_a=np.ones(m - 1),
    scale_eta=np.ones(m - 1),
)

val_scaled_v, grad_logv_costfun_v = costfun(
    x,
    logv[0],
    logv[1],
    logv[2],
    delta_mu,
    delta_a,
    eta / ts,
    fix_logv_alpha=False,
    fix_logv_beta=False,
    fix_logv_tau=False,
    fix_delta_mu=True,
    fix_delta_a=True,
    fix_eta=True,
    scale_sigma_alpha=1.0,
    scale_sigma_beta=1.0,
    scale_sigma_tau=1.0,
    scale_delta_mu=scale_delta_mu,
    scale_delta_a=np.ones(m - 1),
    scale_eta=np.ones(m - 1),
)

val_scaled, grad_logv_costfun = costfun(
    x,
    np.log(np.exp(logv[0]) / scale_sigma_alpha**2),
    np.log(np.exp(logv[1]) / scale_sigma_beta**2),
    np.log(np.exp(logv[2]) / scale_sigma_tau**2),
    delta_mu,
    delta_a,
    eta / ts,
    fix_logv_alpha=False,
    fix_logv_beta=False,
    fix_logv_tau=False,
    fix_delta_mu=True,
    fix_delta_a=True,
    fix_eta=True,
    scale_sigma_alpha=scale_sigma_alpha,
    scale_sigma_beta=scale_sigma_beta,
    scale_sigma_tau=scale_sigma_tau,
    scale_delta_mu=scale_delta_mu,
    scale_delta_a=np.ones(m - 1),
    scale_eta=np.ones(m - 1),
)

grad_logv_nd = nd.Gradient(
    lambda _logv: costfun(
        x,
        np.log(np.exp(_logv[0]) / scale_sigma_alpha**2),
        np.log(np.exp(_logv[1]) / scale_sigma_beta**2),
        np.log(np.exp(_logv[2]) / scale_sigma_tau**2),
        delta_mu,
        delta_a,
        eta / ts,
        fix_logv_alpha=True,
        fix_logv_beta=True,
        fix_logv_tau=True,
        fix_delta_mu=True,
        fix_delta_a=True,
        fix_eta=True,
        scale_sigma_alpha=scale_sigma_alpha,
        scale_sigma_beta=scale_sigma_beta,
        scale_sigma_tau=scale_sigma_tau,
        scale_delta_mu=scale_delta_mu,
        scale_delta_a=scale_delta_a,
        scale_eta=np.ones(m - 1),
    )[0]
)(logv)

print(f"{val =}")
print(f"{val_scaled_v =}")
print(f"{val_scaled =}")
np.stack(
    (
        grad_logv_tdnll,
        grad_logv_costfun_v,
        grad_logv_costfun,
        grad_logv_nd,
    )
).T

In [ ]:
_, grad_delta_a_tdnll = costfun(
    x,
    logv[0],
    logv[1],
    logv[2],
    delta_mu,
    delta_a,
    eta / ts,
    fix_logv_alpha=True,
    fix_logv_beta=True,
    fix_logv_tau=True,
    fix_delta_mu=True,
    fix_delta_a=False,
    fix_eta=True,
    scale_sigma_alpha=1.0,
    scale_sigma_beta=1.0,
    scale_sigma_tau=1.0,
    scale_delta_mu=scale_delta_mu,
    scale_delta_a=np.ones(m - 1),
    scale_eta=np.ones(m - 1),
)

grad_delta_a_nd = nd.Gradient(
    lambda _delta_a: costfun(
        x,
        logv[0],
        logv[1],
        logv[2],
        delta_mu,
        _delta_a,
        eta / ts,
        fix_logv_alpha=True,
        fix_logv_beta=True,
        fix_logv_tau=True,
        fix_delta_mu=True,
        fix_delta_a=True,
        fix_eta=True,
        scale_sigma_alpha=1.0,
        scale_sigma_beta=1.0,
        scale_sigma_tau=1.0,
        scale_delta_mu=scale_delta_mu,
        scale_delta_a=np.ones(m - 1),
        scale_eta=np.ones(m - 1),
    )[0]
)(delta_a)

np.stack((grad_delta_a_tdnll, grad_delta_a_nd)).T

In [ ]:
_, grad_eta_tdnll = costfun(
    x,
    logv[0],
    logv[1],
    logv[2],
    delta_mu,
    delta_a,
    eta / ts,
    fix_logv_alpha=True,
    fix_logv_beta=True,
    fix_logv_tau=True,
    fix_delta_mu=True,
    fix_delta_a=True,
    fix_eta=False,
    scale_sigma_alpha=1.0,
    scale_sigma_beta=1.0,
    scale_sigma_tau=1.0,
    scale_delta_mu=scale_delta_mu,
    scale_delta_a=np.ones(m - 1),
    scale_eta=np.ones(m - 1),
)

grad_eta_nd = nd.Gradient(
    lambda _eta_scaled: costfun(
        x,
        logv[0],
        logv[1],
        logv[2],
        delta_mu,
        delta_a,
        _eta_scaled,
        fix_logv_alpha=True,
        fix_logv_beta=True,
        fix_logv_tau=True,
        fix_delta_mu=True,
        fix_delta_a=True,
        fix_eta=True,
        scale_sigma_alpha=1.0,
        scale_sigma_beta=1.0,
        scale_sigma_tau=1.0,
        scale_delta_mu=scale_delta_mu,
        scale_delta_a=np.ones(m - 1),
        scale_eta=np.ones(m - 1),
    )[0]
)(eta)

np.stack((grad_eta_tdnll, grad_eta_nd)).T

## Estimate noise parameters with revised NLL

In [ ]:
result = thz.noisefit(
    x.T,
    sigma_alpha0=sigma[0],
    sigma_beta0=sigma[1],
    sigma_tau0=sigma[2],
    dt=ts,
    fix_a=False,
    fix_eta=False,
)

In [ ]:
print(result.diagnostic["message"])

In [ ]:
sigma_out = np.array(
    [
        result.noise_model.sigma_alpha,
        result.noise_model.sigma_beta,
        result.noise_model.sigma_tau,
    ]
) * np.sqrt(m / (m - 1))
sigma_err = np.sqrt(
    np.array(
        [result.err_sigma_alpha, result.err_sigma_beta, result.err_sigma_tau]
    )
    * m
    / (m - 1)
)
for sigma_in, sigma_out, err in zip(sigma, sigma_out, sigma_err):
    print(f"Input: {sigma_in:6.4g}\t Output: {sigma_out:6.4g} ± {err:6.4g}")

In [ ]:
plt.plot(result.diagnostic["jac"])
plt.show()

In [ ]:
plt.semilogy(np.diag(result.diagnostic["hess_inv"]))
plt.show()

In [ ]:
np.diag(result.diagnostic["hess_inv"][:3, :3])

In [ ]:
plt.plot(t, np.log10(np.diag(result.diagnostic["hess_inv"])[3 : 3 + n]))
plt.plot(t, mu)
plt.show()

In [ ]:
noise_model_fit = thz.NoiseModel(
    sigma_alpha=result.noise_model.sigma_alpha,
    sigma_beta=result.noise_model.sigma_beta,
    sigma_tau=result.noise_model.sigma_tau,
)

plt.plot(t, result.err_mu)
plt.plot(t, noise_model_fit.amplitude(result.mu))
plt.plot(t, np.std(x, axis=0))
plt.show()

## Repeat fit with amplitudes and delays fixed

In [ ]:
result = thz.noisefit(
    x.T,
    sigma_alpha0=sigma[0],
    sigma_beta0=sigma[1],
    sigma_tau0=sigma[2],
    dt=ts,
    fix_a=True,
    fix_eta=True,
)
print(result.diagnostic["message"])

In [ ]:
sigma_out = np.array(
    [
        result.noise_model.sigma_alpha,
        result.noise_model.sigma_beta,
        result.noise_model.sigma_tau,
    ]
) * np.sqrt(m / (m - 1))
sigma_err = np.sqrt(
    np.array(
        [result.err_sigma_alpha, result.err_sigma_beta, result.err_sigma_tau]
    )
    * m
    / (m - 1)
)
for sigma_in, sigma_out, err in zip(sigma, sigma_out, sigma_err):
    print(f"Input: {sigma_in:6.4g}\t Output: {sigma_out:6.4g} ± {err:6.4g}")

In [ ]:
plt.plot(t, result.err_mu)
plt.plot(t, noise_model_fit.amplitude(result.mu))
plt.plot(t, np.std(x, axis=0))
plt.show()